<a href="https://colab.research.google.com/github/MallikharjunaSakhamuri/EdgeRepresentationDHT/blob/main/EdgeRepresentationDHT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install stellargraph

In [ ]:
!pip install chardet

In [ ]:
import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from stellargraph import StellarGraph

In [ ]:
from sklearn import model_selection
from sklearn.metrics import confusion_matrix,auc,precision_recall_curve,roc_curve
from tensorflow.keras import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import binary_crossentropy,mean_squared_error
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from stellargraph import IndexedArray

In [ ]:
import itertools
import matplotlib.pyplot as plt

In [ ]:
def read_para(path):
    df = pd.read_csv(path,sep='\t',header=None)
    dic = {}
    for i in range(df.shape[0]):
        hla = df[0].iloc[i]
        paratope = df[1].iloc[i]
        try:
            dic[hla] = paratope
        except KeyError:
            dic[hla] = []
            dic[hla].append(paratope)
    return dic

In [ ]:
def dict_inventory(inventory):
    dicA, dicB, dicC = {}, {}, {}
    dic = {'A': dicA, 'B': dicB, 'C': dicC}

    for hla in inventory:
        type_ = hla[4]  # A,B,C
        first2 = hla[6:8]  # 01
        last2 = hla[8:]  # 01
        try:
            dic[type_][first2].append(last2)
        except KeyError:
            dic[type_][first2] = []
            dic[type_][first2].append(last2)

    return dic

In [ ]:
def rescue_unknown_hla(hla, dic_inventory):
    type_ = hla[4]
    first2 = hla[6:8]
    last2 = hla[8:]
    big_category = dic_inventory[type_]
    #print(hla)
    if not big_category.get(first2) == None:
        small_category = big_category.get(first2)
        distance = [abs(int(last2) - int(i)) for i in small_category]
        optimal = min(zip(small_category, distance), key=lambda x: x[1])[0]
        return 'HLA-' + str(type_) + '*' + str(first2) + str(optimal)
    else:
        small_category = list(big_category.keys())
        distance = [abs(int(first2) - int(i)) for i in small_category]
        optimal = min(zip(small_category, distance), key=lambda x: x[1])[0]
        return 'HLA-' + str(type_) + '*' + str(optimal) + str(big_category[optimal][0])

In [ ]:
class Graph_Constructor():

    @staticmethod
    def combinator(pep,hla):
        source = ['p' + str(i+1) for i in range(len(pep))]
        target = ['h' + str(i+1) for i in range(len(hla))]
        return source,target

    @staticmethod
    def numerical(pep,hla,after_pca,embed=12):   # after_pca [21,12]
        pep = pep.replace('X','-').upper()
        hla = hla.replace('X','-').upper()
        feature_array_pep = np.empty([len(pep),embed])
        feature_array_hla = np.empty([len(hla),embed])
        amino = 'ARNDCQEGHILKMFPSTWYV-'
        for i in range(len(pep)):
            feature_array_pep[i,:] = after_pca[amino.index(pep[i]),:]
        for i in range(len(hla)):
            feature_array_hla[i,:] = after_pca[amino.index(hla[i]),:]
        feature_array = np.concatenate([feature_array_pep,feature_array_hla],axis=0)
        #print(feature_array_pep.shape,feature_array_hla.shape,feature_array.shape)
        return feature_array

    @staticmethod
    def unweight_edge(pep,hla,after_pca):
        source,target = Graph_Constructor.combinator(pep,hla)
        combine = list(itertools.product(source,target))
        weight = itertools.repeat(1,len(source)*len(target))
        edges = pd.DataFrame({'source':[item[0] for item in combine],'target':[item[1] for item in combine],'weight':weight})
        feature_array = Graph_Constructor.numerical(pep,hla,after_pca)
        try:nodes = IndexedArray(feature_array,index=source+target)
        except: print(pep,hla,feature_array.shape)
        graph = StellarGraph(nodes,edges,node_type_default='corner',edge_type_default='line')
        return graph

    @staticmethod
    def update_edges_new(pep,hla,edges):

#       print ( "pep", pep)
#       print ( "hla", hla)
#       print ( "edges", edges)

      pep = pep.replace('X','-').upper()
      hla = hla.replace('X','-').upper()

      amino = 'ARNDCQEGHILKMFPSTWYV-'

      edges_weights = pd.read_csv(r'D:\PhD\New_Graph_Model\data\Edge_weights_pca_array_new.txt',sep=' ',header = None)

      # create a dictionary to map amino acid characters to their position in the amino acid string
      amino_dict = {}
      for i, c in enumerate(amino):
        amino_dict[c] = i

      # loop through the rows of the edges dataframe
      for index, row in edges.iterrows():
            # get the source and target nodes from the row
            source = row['source']
            target = row['target']


            # check if source is a peptide or HLA allele
            if source[0] == 'p':
            # get the position of the amino acid in the peptide sequence
                pos = int(source[1:]) - 1

                # get the amino acid character at that position
                amino_char = pep[pos]

                # get the index of the amino acid character in the amino acid string
                amino_pos = amino_dict[amino_char]

            elif source[0] == 'h':
                # get the position of the amino acid in the HLA allele
                pos = int(source[1:]) - 1

                # get the amino acid character at that position
                amino_char = hla[pos]

                # get the index of the amino acid character in the amino acid string
                amino_pos = amino_dict[amino_char]

            # check if target is a peptide or HLA allele
            if target[0] == 'p':
                # get the position of the amino acid in the peptide sequence
                pos = int(target[1:]) - 1

                # get the amino acid character at that position
                amino_char = pep[pos]

                # get the index of the amino acid character in the amino acid string
                amino_pos2 = amino_dict[amino_char]

            elif target[0] == 'h':
                # get the position of the amino acid in the HLA allele
                pos = int(target[1:]) - 1

                # get the amino acid character at that position
                amino_char = hla[pos]

                # get the index of the amino acid character in the amino acid string
                amino_pos2 = amino_dict[amino_char]

                # get the weight value from the weights dataframe
                weight = edges_weights.iloc[amino_pos, amino_pos2]

            # update the weight column in the edges dataframe
            #edges.at[index, 'weight'] = weight
            edges['weight'] = edges['weight'].astype(float)
            edges.at[index, 'weight'] = (weight)

      # Check if source[0] and target[0] are same, if not update weight accordingly
      for index, row in edges.iterrows():
        source, target = row['source'], row['target']
        weight = row['weight']
        mean = np.mean(edges['weight'])
        std = np.std(edges['weight'])

        if source[0] != target[0] and (weight > (mean + std) or weight < (mean - std)):
            edges.at[index, 'weight'] = -1

      return edges

    @staticmethod
    def weight_anchor_edge(pep,hla,after_pca):
        source, target = Graph_Constructor.combinator(pep, hla)
        combine = list(itertools.product(source, target))
        weight = itertools.repeat(1, len(source) * len(target))
        edges = pd.DataFrame({'source': [item[0] for item in combine], 'target': [item[1] for item in combine], 'weight': weight})
        for i in range(edges.shape[0]):
            col1 = edges.iloc[i]['source']
            col2 = edges.iloc[i]['target']
            col3 = edges.iloc[i]['weight']
            if col1 == 'a2' or col1 == 'a9' or col1 ==  'a10':
                edges.iloc[i]['weight'] = 1.5
        feature_array = Graph_Constructor.numerical(pep, hla, after_pca)
        nodes = IndexedArray(feature_array, index=source + target)
        graph = StellarGraph(nodes, edges, node_type_default='corner', edge_type_default='line')
        return graph

    @staticmethod
    def intra_and_inter(pep,hla,after_pca):
        source, target = Graph_Constructor.combinator(pep, hla)
        combine = list(itertools.product(source, target))
        weight = itertools.repeat(2, len(source) * len(target))
        edges_inter = pd.DataFrame({'source': [item[0] for item in combine], 'target': [item[1] for item in combine], 'weight': weight})
        intra_pep = list(itertools.combinations(source,2))
        intra_hla = list(itertools.combinations(target,2))
        intra = intra_pep + intra_hla
        weight = itertools.repeat(1,len(intra))
        edges_intra = pd.DataFrame({'source':[item[0] for item in intra],'target':[item[1] for item in intra],'weight':weight})
        edges = pd.concat([edges_inter,edges_intra])
        edges = edges.set_index(pd.Index(np.arange(edges.shape[0])))

        # Updating Edge weights
        edges = Graph_Constructor.update_edges_new(pep,hla,edges)

        # Dual Hyper Transfer
        #New_nodes,new_Edges = Graph_Constructor.DHT(source,target,edges,feature_array)

        feature_array = Graph_Constructor.numerical(pep, hla, after_pca)
        nodes = IndexedArray(feature_array, index=source + target)
#         print('Nodes Type',type(nodes))
#         print('Edges Type',type(edges))
#         print ("nodes, edges:",nodes, edges)
        graph = StellarGraph(nodes, edges, node_type_default='corner', edge_type_default='line')
        return graph

    @staticmethod
    def entrance_v1(df,after_pca,hla_dic,dic_inventory):
        graphs = []
        graph_labels = []
        for i in range(df.shape[0]):
            print(i)
            pep = df['peptide'].iloc[i]
            try:
                hla = hla_dic[df['HLA'].iloc[i]]
            except KeyError:
                hla = hla_dic[rescue_unknown_hla(df['HLA'].iloc[i],dic_inventory)]
            label = df['immunogenicity'].iloc[i]
            #if label != 'Negative': label = 0
            #else: label = 1
            #graph = Graph_Constructor.unweight_edge(pep,hla,after_pca)
            #graph = Graph_Constructor.unweight_edge(pep,hla,after_pca)
            graph = Graph_Constructor.intra_and_inter(pep,hla,after_pca)
            graphs.append(graph)
            graph_labels.append(label)
        graph_labels = pd.Series(graph_labels)
        return graphs,graph_labels

    @staticmethod
    def entrance(df, after_pca, hla_dic, dic_inventory):
        graphs = []
        graph_labels = []
        for i in range(df.shape[0]):
            print(i)
            pep = df['peptide'].iloc[i]
            try:
                hla = hla_dic[df['HLA'].iloc[i]]
            except KeyError:
                hla = hla_dic[rescue_unknown_hla(df['HLA'].iloc[i], dic_inventory)]
            label = df['immunogenicity'].iloc[i]

            # Construct the hypergraph
            graph = HyperGraph_Constructor.hypergraph(pep, hla, after_pca)

            graphs.append(graph)
            graph_labels.append(label)
        graph_labels = pd.Series(graph_labels)
        return graphs, graph_labels

In [ ]:
class HyperGraph_Constructor_v1(Graph_Constructor):

    @staticmethod
    def combinator_hyperedge(pep, hla):
        # Create source-target pairs for hyperedges
        source = ['p' + str(i+1) for i in range(len(pep))]
        target = ['h' + str(i+1) for i in range(len(hla))]
        # In hypergraphs, one edge can link multiple nodes, so return both sets together
        return source, target

    @staticmethod
    def hypergraph(pep, hla, after_pca):
        source, target = HyperGraph_Constructor.combinator_hyperedge(pep, hla)
        combine = list(itertools.product(source, target))  # Hyperedges can be more complex
        weight = itertools.repeat(1, len(source) * len(target))

        edges = pd.DataFrame({'source': [item[0] for item in combine],
                              'target': [item[1] for item in combine],
                              'weight': weight})

        feature_array = HyperGraph_Constructor.numerical(pep, hla, after_pca)
        nodes = IndexedArray(feature_array, index=source + target)
        hypergraph = StellarGraph(nodes, edges, node_type_default='corner', edge_type_default='line')

        return hypergraph

In [ ]:
class HyperGraph_Constructor(Graph_Constructor):

    @staticmethod
    def combinator_hyperedge(pep, hla):
        """
        Create source-target pairs for hyperedges where each node (from peptide and HLA)
        is connected to each other in a bipartite-like graph.
        """
        source = ['p' + str(i+1) for i in range(len(pep))]
        target = ['h' + str(i+1) for i in range(len(hla))]
        return source, target

    @staticmethod
    def hypergraph(pep, hla, after_pca):
        """
        Construct a hypergraph from peptide and HLA sequences with associated numerical features.
        """
        # Get source and target nodes
        source, target = HyperGraph_Constructor.combinator_hyperedge(pep, hla)

        # Create product of all possible source-target pairs to form hyperedges
        combine = list(itertools.product(source, target))

        # Assign a weight of 1 to all hyperedges (this can be adapted for variable weighting)
        weights = [1] * len(combine)  # Replace itertools.repeat for clarity

        # Create edges DataFrame with source-target pairs and their weights
        edges = pd.DataFrame({
            'source': [item[0] for item in combine],
            'target': [item[1] for item in combine],
            'weight': weights
        })

        # Generate node features from peptide and HLA sequences
        feature_array = HyperGraph_Constructor.numerical(pep, hla, after_pca)

        # Combine source and target nodes to form the node set
        node_index = source + target

        # Create a dictionary where node index maps to their feature vectors
        nodes = pd.DataFrame(feature_array, index=node_index)

        # Create a StellarGraph object with nodes and edges
        hypergraph = StellarGraph(nodes=nodes, edges=edges)

        return hypergraph


In [ ]:
class HyperGraph_Constructor_DHT(Graph_Constructor):

    @staticmethod
    def combinator_hyperedge(pep, hla):
        """
        Create source-target pairs for hyperedges where each node (from peptide and HLA)
        is connected to each other in a bipartite-like graph.
        """
        source = ['p' + str(i+1) for i in range(len(pep))]
        target = ['h' + str(i+1) for i in range(len(hla))]
        return source, target

    @staticmethod
    def dht_incidence_matrix(edges_df, node_index):
        """
        Create the incidence matrix for the dual hypergraph transformation.
        Swaps the role of nodes and edges.
        """
        incidence_matrix = np.zeros((len(node_index), len(edges_df)))

        # For each edge, set the incidence of its source and target nodes
        for i, (source, target) in enumerate(zip(edges_df['source'], edges_df['target'])):
            incidence_matrix[node_index.index(source), i] = 1
            incidence_matrix[node_index.index(target), i] = 1

        return incidence_matrix.T  # Return the transpose for the dual form

    @staticmethod
    def dual_hypergraph(pep, hla, after_pca):
        """
        Construct the dual hypergraph where edges are treated as nodes, and nodes as hyperedges.
        """
        source, target = HyperGraph_Constructor_DHT.combinator_hyperedge(pep, hla)
        combine = list(itertools.product(source, target))
        weights = [1] * len(combine)  # Assign weight of 1 to all edges

        # Create edges DataFrame with source-target pairs and weights
        edges = pd.DataFrame({
            'source': [item[0] for item in combine],
            'target': [item[1] for item in combine],
            'weight': weights
        })

        # Generate node features from peptide and HLA sequences
        feature_array = HyperGraph_Constructor_DHT.numerical(pep, hla, after_pca)
        node_index = source + target

        # Apply Dual Hypergraph Transformation (DHT) - swap the role of nodes and edges
        incidence_matrix = HyperGraph_Constructor_DHT.dht_incidence_matrix(edges, node_index)

        # Create a StellarGraph object with nodes and incidence matrix (now hyperedges)
        nodes_df = pd.DataFrame(feature_array, index=node_index)
        dual_hypergraph = StellarGraph(nodes=nodes_df, edges=edges)

        return dual_hypergraph


In [ ]:
class HyperGraph_Constructor_DHT_Cluster(HyperGraph_Constructor_DHT):

    @staticmethod
    def entrance(df, after_pca, hla_dic, dic_inventory, n_clusters=5):
        """
        Custom entrance method for HyperGraph_Constructor_DHT_Cluster, which constructs dual hypergraphs
        with edge clustering applied for graph-level classification.

        Args:
        - df: DataFrame containing HLA and peptide data.
        - after_pca: PCA-transformed feature data for amino acids.
        - hla_dic: Dictionary mapping HLA alleles to their pseudo sequences.
        - dic_inventory: Dictionary containing inventory of HLA alleles.
        - n_clusters: Number of clusters for edge clustering (default: 5).

        Returns:
        - graphs: List of dual hypergraphs with clustered edges.
        - graph_labels: Corresponding labels for graph-level classification.
        """
        graphs = []
        graph_labels = []

        # Loop over each sample in the DataFrame
        for i in range(df.shape[0]):
            print(f"Processing sample {i + 1}/{df.shape[0]}")  # Debug print to track progress

            # Get the peptide and HLA information
            pep = df['peptide'].iloc[i]
            try:
                hla = hla_dic[df['HLA'].iloc[i]]
            except KeyError:
                hla = hla_dic[rescue_unknown_hla(df['HLA'].iloc[i], dic_inventory)]

            # Get the label (immunogenicity) for graph classification
            label = df['immunogenicity'].iloc[i]

            # Construct the dual hypergraph with clustering applied
            graph = HyperGraph_Constructor_DHT_Cluster.dual_hypergraph_cluster(pep, hla, after_pca, n_clusters=n_clusters)

            # Append the constructed graph and its label to the lists
            graphs.append(graph)
            graph_labels.append(label)

        # Convert graph_labels to a pandas Series for consistency
        graph_labels = pd.Series(graph_labels)

        return graphs, graph_labels

    @staticmethod
    def generate_edge_features(edges_df, node_features):
        """
        Generate edge features by combining the source and target node features.
        This can be done by concatenating or averaging the node features of the source and target nodes.

        Args:
        - edges_df: DataFrame with source and target node identifiers.
        - node_features: DataFrame or numpy array containing the node features.

        Returns:
        - edge_features: Features generated for each edge by combining the source and target node features.
        """
        edge_features = []

        # Loop over each edge to combine the source and target node features
        for idx, row in edges_df.iterrows():
            source_node = row['source']
            target_node = row['target']

            # Get the node features for the source and target
            source_feature = node_features.loc[source_node].values
            target_feature = node_features.loc[target_node].values

            # Combine source and target features (e.g., concatenation, mean, etc.)
            combined_feature = np.concatenate([source_feature, target_feature])

            # Append the combined feature to the list
            edge_features.append(combined_feature)

        # Convert the list to a numpy array for clustering
        edge_features = np.array(edge_features)

        return edge_features

    @staticmethod
    def hypercluster(edges_df, edge_features, n_clusters=5):
        """
        Perform clustering of edges to aggregate similar edges into clusters.
        This reduces the graph complexity by grouping edges based on their similarity.
        """
        from sklearn.cluster import KMeans

        # Perform KMeans clustering on the edge features
        kmeans = KMeans(n_clusters=n_clusters)
        clusters = kmeans.fit_predict(edge_features)

        # Add cluster labels to the edges DataFrame
        edges_df['cluster'] = clusters

        # Check if the 'weight' column exists, if not, assign a default weight
        if 'weight' not in edges_df.columns:
            edges_df['weight'] = 1.0  # Assign default weight

        # Aggregate edges within each cluster (e.g., mean weight, or other aggregation methods)
        clustered_edges = edges_df.groupby('cluster').agg({
            'source': 'first',
            'target': 'first',
            'weight': 'mean'  # Use 'mean' as the aggregation method for weights
        }).reset_index(drop=True)

#         print("clustered_edges.head():", clustered_edges.head())  # Debug output

        return clustered_edges

    @staticmethod
    def dual_hypergraph_cluster(pep, hla, after_pca, n_clusters=5):
        """
        Construct the dual hypergraph and apply clustering on edge representations.
        """
        # Construct the dual hypergraph as before
        dual_hypergraph = HyperGraph_Constructor_DHT.dual_hypergraph(pep, hla, after_pca)
        edges_df = dual_hypergraph.edges()

        # Ensure that edges_df is a DataFrame
        edges_df = pd.DataFrame(edges_df, columns=['source', 'target'])

        # Get the node features from the graph
        node_features = pd.DataFrame(dual_hypergraph.node_features(), index=dual_hypergraph.nodes())

        # Generate edge features by combining the source and target node features
        edge_features = HyperGraph_Constructor_DHT_Cluster.generate_edge_features(edges_df, node_features)

        # Perform clustering on the edge features
        clustered_edges = HyperGraph_Constructor_DHT_Cluster.hypercluster(edges_df, edge_features, n_clusters=n_clusters)

        # Rebuild the hypergraph with clustered edges
        hypergraph_clustered = StellarGraph(
            nodes=node_features,  # Correctly formatted node features
            edges=clustered_edges
        )

        return hypergraph_clustered




In [ ]:
class HyperGraph_Constructor_DHT_HyperDrop(HyperGraph_Constructor_DHT):

    @staticmethod
    def hyperdrop(edges_df, edge_features, top_k=0.5):
        """
        Perform edge dropping based on the importance score of edges.
        This will drop the bottom (1 - top_k)% of edges and keep the top_k% for classification.
        """
        # Compute scores for each edge (e.g., L2 norm or any other metric)
        edge_scores = np.linalg.norm(edge_features, axis=1)

        # Sort edges by their scores in descending order and keep top_k percentage
        num_edges_to_keep = int(top_k * len(edge_scores))
        top_edge_indices = np.argsort(edge_scores)[-num_edges_to_keep:]

        # Convert edges_df from list of tuples to DataFrame
        edges_df = pd.DataFrame(edges_df, columns=['source', 'target'])

        # Add a default 'weight' column if it doesn't exist
        edges_df['weight'] = 1  # Assign default weight

        # Keep only the top_k percentage of edges
        edges_dropped = edges_df.iloc[top_edge_indices]

        print("edges_dropped.head():", edges_dropped.head())  # Check the result

        return edges_dropped

    @staticmethod
    def dual_hypergraph_hyperdrop(pep, hla, after_pca, top_k=0.5):
        """
        Construct the dual hypergraph and apply edge dropping (HyperDrop) on edge representations.
        """
        # Construct the dual hypergraph as before
        dual_hypergraph = HyperGraph_Constructor_DHT.dual_hypergraph(pep, hla, after_pca)
        edges_df = dual_hypergraph.edges()

        # Apply HyperDrop to drop irrelevant edges
        edge_features = dual_hypergraph.node_features()
        dropped_edges = HyperGraph_Constructor_DHT_HyperDrop.hyperdrop(edges_df, edge_features, top_k=top_k)

        # Ensure node features are in the correct format (DataFrame or dict)
        node_features = pd.DataFrame(dual_hypergraph.node_features(), index=dual_hypergraph.nodes())

        # Ensure correct DataFrame structure
        assert 'source' in dropped_edges.columns, "Missing source column in edges_dropped"
        assert 'target' in dropped_edges.columns, "Missing target column in edges_dropped"

        # Rebuild the hypergraph with only the top_k% edges
        hypergraph_dropped = StellarGraph(
            nodes=node_features,  # Correctly formatted node features
            edges=dropped_edges
        )

        return hypergraph_dropped

    @staticmethod
    def entrance(df, after_pca, hla_dic, dic_inventory, top_k=0.5):
        graphs = []
        graph_labels = []
        for i in range(df.shape[0]):
            pep = df['peptide'].iloc[i]
            try:
                hla = hla_dic[df['HLA'].iloc[i]]
            except KeyError:
                hla = hla_dic[rescue_unknown_hla(df['HLA'].iloc[i], dic_inventory)]
            label = df['immunogenicity'].iloc[i]

            # Construct the dual hypergraph with HyperDrop applied
            graph = HyperGraph_Constructor_DHT_HyperDrop.dual_hypergraph_hyperdrop(pep, hla, after_pca, top_k=top_k)

            graphs.append(graph)
            graph_labels.append(label)

        graph_labels = pd.Series(graph_labels)
        return graphs, graph_labels

In [ ]:
def train_fold(model, train_gen, test_gen, es, epochs):
    history = model.fit(
        train_gen, epochs=epochs, validation_data=test_gen, verbose=2, callbacks=[es],)
    # calculate performance on the test data and return along with history
    test_metrics = model.evaluate(test_gen, verbose=0)
    test_acc = test_metrics[model.metrics_names.index("acc")]
    return history, test_acc

In [ ]:
# def get_generators(train_index, test_index, graph_labels, batch_size):
#     train_gen = generator.flow(
#         train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size)
#     test_gen = generator.flow(
#         test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size)

#     return train_gen, test_gen


def get_generators(train_index, val_index, test_index, graph_labels, batch_size):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size
    )
    val_gen = generator.flow(
        val_index, targets=graph_labels.iloc[val_index].values, batch_size=batch_size
    )
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size
    )
    return train_gen, val_gen, test_gen


In [ ]:
def draw_ROC(y_true,y_pred):

    fpr,tpr,_ = roc_curve(y_true,y_pred,pos_label=1)
    area_mine = auc(fpr,tpr)
    fig = plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % area_mine)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
def draw_PR(y_true,y_pred):
    precision,recall,_ = precision_recall_curve(y_true,y_pred,pos_label=1)
    area_PR = auc(recall,precision)
    baseline = np.sum(np.array(y_true) == 1) / len(y_true)

    plt.figure()
    lw = 2
    plt.plot(recall,precision, color='darkorange',
            lw=lw, label='PR curve (area = %0.2f)' % area_PR)
    plt.plot([0, 1], [baseline, baseline], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('PR curve example')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
def draw_history(history):
    plt.subplot(211)
    plt.title('Loss')
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.legend()
    # plot accuracy during training
    plt.subplot(212)
    plt.title('Accuracy')
    plt.plot(history.history['acc'], label='train')
    plt.plot(history.history['val_acc'], label='validation')
    plt.legend()
    plt.show()

In [ ]:
def hla_df_to_dic(hla):
    dic = {}
    for i in range(hla.shape[0]):
        col1 = hla['HLA'].iloc[i]  # HLA allele
        col2 = hla['pseudo'].iloc[i]  # pseudo sequence
        dic[col1] = col2
    return dic

In [ ]:
def retain_910(ori):
    cond = []
    for i in range(ori.shape[0]):
        peptide = ori['peptide'].iloc[i]
        if len(peptide) == 9 or len(peptide) == 10:
            cond.append(True)
        else:
            cond.append(False)
    data = ori.loc[cond]
    data = data.set_index(pd.Index(np.arange(data.shape[0])))
    return data

In [ ]:
ori_train = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\remove0123_sample100.csv')
# ori_train = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\remove0123_sample100_Test.csv')
hla = pd.read_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\hla2paratopeTable_aligned.txt',sep='\t')
after_pca = np.loadtxt('D:\\PhD\\GraphImmuno_NewResults\\data\\after_pca.txt')

hla_dic = hla_df_to_dic(hla)
inventory = list(hla_dic.keys())
dic_inventory = dict_inventory(inventory)

ori_train['immunogenicity'], ori_train['potential'] = ori_train['potential'], ori_train['immunogenicity']

In [ ]:
# Create graphs and labels using the new HyperGraph_Constructor
#graphs, graph_labels = HyperGraph_Constructor_DHT.entrance(ori_train, after_pca, hla_dic, dic_inventory)
graphs, graph_labels = HyperGraph_Constructor_DHT_HyperDrop.entrance(ori_train, after_pca, hla_dic, dic_inventory, top_k=0.5)
# graphs, graph_labels = HyperGraph_Constructor_DHT_Cluster.entrance(ori_train, after_pca, hla_dic, dic_inventory, n_clusters=5)

# Initialize the generator for dual hypergraphs
generator = PaddedGraphGenerator(graphs=graphs)

In [ ]:
graph_labels

In [ ]:
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

In [ ]:
ori_train1 = ori_train

In [ ]:
ori_train1.loc[ori_train1["potential"] != "Negative", "potential"] = 'Positive'
ori_train1['potential'] = ori_train1['potential'].astype('category')
ori_train1['potential'] = pd.factorize(ori_train1['potential'])[0]

In [ ]:
graph_labels = ori_train1['potential']

In [ ]:
graph_labels.value_counts().to_frame()

In [ ]:
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# Split train and test set
# train_index, test_index = train_test_split(graph_labels.index, train_size=0.5, stratify=graph_labels)

# First split into train and temp (which will be split into validation and test)
train_index, temp_index = train_test_split(
    graph_labels.index, train_size=0.6, stratify=graph_labels
)

# Now split temp into validation and test
val_index, test_index = train_test_split(
    temp_index, train_size=0.5, stratify=graph_labels.loc[temp_index]
)

# Define the PaddedGraphGenerator for the StellarGraph model
generator = PaddedGraphGenerator(graphs=graphs)

# Define the GCN model
gc_model = GCNSupervisedGraphClassification(
    layer_sizes=[64, 64],
    activations=["relu", "relu"],
    generator=generator,
    dropout=0.2,
)

# Build the model structure
x_inp, x_out = gc_model.in_out_tensors()

# Add Dense layers
predictions = Dense(units=32, activation="relu")(x_out)
predictions = Dense(units=16, activation="relu")(predictions)
predictions = Dense(units=1, activation="sigmoid")(predictions)  # Sigmoid for binary classification

# Define the final model
model = Model(inputs=x_inp, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Define the training and testing generators
# def get_generators(train_index, test_index, graph_labels, batch_size):
#     train_gen = generator.flow(train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size)
#     test_gen = generator.flow(test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size)
#     return train_gen, test_gen

def get_generators(train_index, val_index, test_index, graph_labels, batch_size):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, batch_size=batch_size
    )
    val_gen = generator.flow(
        val_index, targets=graph_labels.iloc[val_index].values, batch_size=batch_size
    )
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values, batch_size=batch_size
    )
    return train_gen, val_gen, test_gen




In [ ]:
epochs = 100
# Get the training and testing generators
# train_gen, test_gen = get_generators(train_index, test_index, graph_labels, batch_size=50)

train_gen, val_gen, test_gen = get_generators(train_index, val_index, test_index, graph_labels, batch_size=50)

# Define EarlyStopping callback
es = EarlyStopping(
    monitor='val_accuracy',  # Monitor validation accuracy instead of loss
    patience=10,             # Number of epochs with no improvement before stopping
    restore_best_weights=True
)

# Train the model with EarlyStopping
# history = model.fit(
#     train_gen,
#     epochs=100,
#     validation_data=test_gen,
#     verbose=2,
#     callbacks=[es],  # EarlyStopping callback
# )

# Train the model with EarlyStopping on validation data
history = model.fit(
    train_gen,
    epochs=100,
    validation_data=val_gen,
    verbose=2,
    callbacks=[es],  # EarlyStopping callback
)

# # Evaluate the model on the test set
# test_metrics = model.evaluate(test_gen, verbose=0)

# # Print test metrics
# print("\nTest Set Metrics:")
# for name, val in zip(model.metrics_names, test_metrics):
#     print(f"\t{name}: {val:.4f}")

# Evaluate the model on the test set
test_metrics = model.evaluate(test_gen, verbose=0)

# Print test metrics
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print(f"\t{name}: {val:.4f}")


In [ ]:
test_metrics = model.evaluate(test_gen)
print("\nTest Set Metrics:")
for name, val in zip(model.metrics_names, test_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
!pip install seaborn

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc, precision_recall_curve, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import seaborn as sns
import matplotlib.pyplot as plt

# Get the true labels and predicted probabilities from the test generator
y_true = np.concatenate([y for _, y in test_gen])  # True labels from the test generator
y_pred_proba = model.predict(test_gen).ravel()     # Predicted probabilities from the model

# Convert predicted probabilities to binary predictions (threshold 0.5)
y_pred = (y_pred_proba > 0.5).astype(int)

In [ ]:
# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()

# Plot ROC curve
def plot_roc_curve(y_true, y_pred_proba):
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

# Plot Precision-Recall curve
def plot_precision_recall_curve(y_true, y_pred_proba):
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = auc(recall, precision)

    plt.figure()
    plt.plot(recall, precision, color='darkorange', lw=2, label='PR curve (area = %0.2f)' % pr_auc)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")
    plt.show()

# Plot training history
def plot_metrics(history):
    plt.figure(figsize=(14, 5))

    # Loss Plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='train')
    plt.plot(history.history['val_loss'], label='validation')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Accuracy Plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='train')
    plt.plot(history.history['val_accuracy'], label='validation')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

In [ ]:
# Plot confusion matrix
plot_confusion_matrix(y_true, y_pred)

# Print classification report
print(classification_report(y_true, y_pred))



In [ ]:
# Plot ROC Curve
plot_roc_curve(y_true, y_pred_proba)


In [ ]:

# Plot Precision-Recall Curve
plot_precision_recall_curve(y_true, y_pred_proba)


In [ ]:

# Plot training history (accuracy and loss)
plot_metrics(history)

In [ ]:
from sklearn.metrics import matthews_corrcoef, classification_report, confusion_matrix, roc_curve, auc, precision_recall_curve
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the MCC value
mcc = matthews_corrcoef(y_true, y_pred)
print(f"Matthews Correlation Coefficient (MCC): {mcc:.4f}")

# Display other classification metrics (optional)
print("\nClassification Report:")
print(classification_report(y_true, y_pred))

# Plot confusion matrix
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix')
    plt.show()

# Plot ROC curve
def plot_roc_curve(y_true, y_pred_proba):
    fpr, tpr, _ = roc_curve(y_true, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")
    plt.show()

# Plot Precision-Recall curve
def plot_precision_recall_curve(y_true, y_pred_proba):
    precision, recall, _ = precision_recall_curve(y_true, y_pred_proba)
    pr_auc = auc(recall, precision)

    plt.figure()
    plt.plot(recall, precision, color='darkorange', lw=2, label='PR curve (area = %0.2f)' % pr_auc)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc="lower right")
    plt.show()

# Example usage after the classification task
# Plot confusion matrix
plot_confusion_matrix(y_true, y_pred)

# Print classification report and MCC value
print(classification_report(y_true, y_pred))
print(f"\nMatthews Correlation Coefficient (MCC): {mcc:.4f}")

# Plot ROC Curve
plot_roc_curve(y_true, y_pred_proba)

# Plot Precision-Recall curve
plot_precision_recall_curve(y_true, y_pred_proba)


In [ ]:
print(f"\nMatthews Correlation Coefficient (MCC): {mcc:.4f}")

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

class HyperGraphClustering:
    def __init__(self, n_clusters=5):
        self.n_clusters = n_clusters

    def apply_kmeans(self, node_features):
        """
        Apply KMeans clustering on node features.

        Args:
        - node_features: The features of the graph nodes.

        Returns:
        - clusters: An array of cluster labels assigned to each node.
        """
        kmeans = KMeans(n_clusters=self.n_clusters)
        clusters = kmeans.fit_predict(node_features)
        return clusters

    def analyze_clusters(self, clusters):
        """
        Analyze the clusters by showing the number of nodes in each cluster.

        Args:
        - clusters: Cluster labels assigned to each node.
        """
        unique, counts = np.unique(clusters, return_counts=True)
        cluster_info = dict(zip(unique, counts))
        print("Cluster Distribution:")
        for cluster_id, size in cluster_info.items():
            print(f"Cluster {cluster_id}: {size} nodes")

    def visualize_clusters(self, graph, clusters):
        """
        Visualize the clusters in a 2D space (using PCA for dimensionality reduction).

        Args:
        - graph: The input hypergraph (StellarGraph object).
        - clusters: Cluster labels assigned to each node.
        """
        node_features = graph.node_features()
        pca = PCA(n_components=2)
        reduced_features = pca.fit_transform(node_features)

        plt.figure(figsize=(10, 8))
        sns.scatterplot(
            x=reduced_features[:, 0],
            y=reduced_features[:, 1],
            hue=clusters,
            palette="viridis",
            s=100,
            alpha=0.8
        )
        plt.title("Graph Clustering Visualization (PCA)")
        plt.show()

# Example usage with your hypergraph:

# Initialize the clustering model
n_clusters = 5
graph_clustering = HyperGraphClustering(n_clusters=n_clusters)

# Get node features from the graph
node_features = [graph.node_features() for graph in graphs]  # Assuming a list of graphs

# Flatten node features for all graphs
node_features = np.vstack(node_features)

# Step 2: Apply KMeans Clustering on Node Features
clusters = graph_clustering.apply_kmeans(node_features)

# Step 3: Analyze the Clusters
graph_clustering.analyze_clusters(clusters)

# Step 4: Visualize the Clusters (optional, for a single graph)
# You can pass one of the graphs and the cluster labels
graph_clustering.visualize_clusters(graphs[0], clusters[:len(graphs[0].nodes())])


In [ ]:
# Create the hypergraphs using the existing workflow
# graphs, graph_labels = HyperGraph_Constructor_DHT_Cluster.entrance(ori_train, after_pca, hla_dic, dic_inventory, n_clusters=5)

# Initialize the clustering class
graph_clustering = HyperGraphClustering(n_clusters=5)

# Get node features for all graphs (assuming each graph has node features)
node_features = [graph.node_features() for graph in graphs]

# Flatten node features for all graphs
node_features = np.vstack(node_features)

# Apply KMeans clustering to the node features
clusters = graph_clustering.apply_kmeans(node_features)

# Analyze the clusters (print number of nodes in each cluster)
graph_clustering.analyze_clusters(clusters)

# Visualize the clusters for a single graph (optional)
graph_clustering.visualize_clusters(graphs[0], clusters[:len(graphs[0].nodes())])


In [ ]:
# Validation on external dataset Dengue

In [ ]:
# ori_train = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\remove0123_sample100.csv')
ori_train_Dengue = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\dengue_test.csv')
ori_train_Dengue_bk = ori_train_Dengue
hla = pd.read_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\hla2paratopeTable_aligned.txt',sep='\t')
after_pca = np.loadtxt('D:\\PhD\\GraphImmuno_NewResults\\data\\after_pca.txt')

hla_dic = hla_df_to_dic(hla)
inventory = list(hla_dic.keys())
dic_inventory = dict_inventory(inventory)

# ori_train['immunogenicity'], ori_train['potential'] = ori_train_external['potential'], ori_train_external['immunogenicity']

In [ ]:
# Create graphs and labels using the new HyperGraph_Constructor
#graphs, graph_labels = HyperGraph_Constructor_DHT.entrance(ori_train, after_pca, hla_dic, dic_inventory)
#graphs, graph_labels = HyperGraph_Constructor_DHT_HyperDrop.entrance(ori_train, after_pca, hla_dic, dic_inventory, top_k=0.5)
external_graphs, external_graph_labels = HyperGraph_Constructor_DHT_Cluster.entrance(ori_train_Dengue, after_pca, hla_dic, dic_inventory, n_clusters=5)

# Initialize the generator for dual hypergraphs
generator = PaddedGraphGenerator(graphs=graphs)

In [ ]:
# graph_labels = pd.get_dummies(graph_labels, drop_first=True)
# ori_train1 = ori_train
# ori_train1.loc[ori_train1["potential"] != "Negative", "potential"] = 'Positive'
# ori_train1['potential'] = ori_train1['potential'].astype('category')
# ori_train1['potential'] = pd.factorize(ori_train1['potential'])[0]
graph_labels = ori_train_Dengue['immunogenicity']
graph_labels.value_counts().to_frame()

In [ ]:
len(external_graph_labels)

In [ ]:
# ori_train_Dengue

In [ ]:
# ori_train_Dengue_bk['immunogenicity']

In [ ]:
assert len(external_graphs) == len(external_graph_labels), "Mismatch in number of graphs and labels."

In [ ]:
# Step 1: Load external datasets
from stellargraph import StellarGraph
import pandas as pd
import numpy as np

# Ensure data integrity
assert len(external_graphs) == len(external_graph_labels), "Mismatch in graphs and labels."

# Step 2: Preprocess external graphs (if necessary)
node_feature_size = graphs[0].node_features().shape[1]

def preprocess_graph(graph):
    features = graph.node_features()
    if features.shape[1] != node_feature_size:
        padded_features = np.zeros((features.shape[0], node_feature_size))
        padded_features[:, :features.shape[1]] = features
        graph = StellarGraph(graph.to_networkx(), node_features=padded_features)
    return graph

external_graphs = [preprocess_graph(g) for g in external_graphs]

# Step 3: Create data generators
from stellargraph.mapper import PaddedGraphGenerator

external_generator = PaddedGraphGenerator(graphs=external_graphs)
external_index = np.arange(len(external_graphs))

external_gen = external_generator.flow(
    external_index,
    targets=external_graph_labels.values,
    batch_size=50,
    symmetric_normalization=False,
)

# Step 4: Predict on external data
external_predictions_proba = model.predict(external_gen).ravel()

# Adjust the threshold based on precision-recall curve
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(external_graph_labels.values, external_predictions_proba)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold:.4f}")

# Apply the optimal threshold
external_predictions = (external_predictions_proba > optimal_threshold).astype(int)

# Step 5: Compute and display metrics
# Step 5: Compute and display metrics
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, average_precision_score

external_y_true = external_graph_labels.values

# Check if more than one class is present
unique_classes = np.unique(external_y_true)
if len(unique_classes) > 1:
    # Compute ROC AUC and PR AUC
    external_roc_auc = roc_auc_score(external_y_true, external_predictions_proba)
    external_pr_auc = average_precision_score(external_y_true, external_predictions_proba)
else:
    external_roc_auc = None
    external_pr_auc = None
    print("Cannot compute ROC AUC or PR AUC - only one class present in y_true.")

# Compute other metrics
external_accuracy = accuracy_score(external_y_true, external_predictions)

# Handle MCC similarly
if len(unique_classes) > 1:
    external_mcc = matthews_corrcoef(external_y_true, external_predictions)
else:
    external_mcc = None
    print("Cannot compute Matthews Correlation Coefficient (MCC) - only one class present in y_true.")

print(f"Accuracy on External Data: {external_accuracy:.4f}")
if external_mcc is not None:
    print(f"Matthews Correlation Coefficient (MCC): {external_mcc:.4f}")
if external_roc_auc is not None:
    print(f"ROC AUC Score: {external_roc_auc:.4f}")
if external_pr_auc is not None:
    print(f"Precision-Recall AUC: {external_pr_auc:.4f}")

# Classification report
print("\nClassification Report on External Data:")
print(classification_report(external_y_true, external_predictions, zero_division=0))

# Proceed with confusion matrix and other available metrics
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(5, 4))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['Predicted Class 1'],
        yticklabels=['Actual Class 1']
    )
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix on External Data')
    plt.show()

# Plot the confusion matrix
plot_confusion_matrix(external_y_true, external_predictions)


# Plot ROC Curve
def plot_roc_curve(y_true, y_proba):
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve on External Data')
    plt.legend(loc='lower right')
    plt.show()

plot_roc_curve(external_y_true, external_predictions_proba)

# Plot Precision-Recall Curve
def plot_precision_recall_curve(y_true, y_proba):
    from sklearn.metrics import precision_recall_curve, auc
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    pr_auc = auc(recall, precision)

    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AUC = {pr_auc:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve on External Data')
    plt.legend(loc='lower left')
    plt.show()

plot_precision_recall_curve(external_y_true, external_predictions_proba)


In [ ]:
#Cell data check

In [ ]:

# ori_train = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\remove0123_sample100.csv')
ori_train_Dengue = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\deephlapan_result_cell.csv')
ori_train_Dengue_bk = ori_train_Dengue
hla = pd.read_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\hla2paratopeTable_aligned.txt',sep='\t')
after_pca = np.loadtxt('D:\\PhD\\GraphImmuno_NewResults\\data\\after_pca.txt')

hla_dic = hla_df_to_dic(hla)
inventory = list(hla_dic.keys())
dic_inventory = dict_inventory(inventory)

# ori_train['immunogenicity'], ori_train['potential'] = ori_train_external['potential'], ori_train_external['immunogenicity']

# Create graphs and labels using the new HyperGraph_Constructor
#graphs, graph_labels = HyperGraph_Constructor_DHT.entrance(ori_train, after_pca, hla_dic, dic_inventory)
#graphs, graph_labels = HyperGraph_Constructor_DHT_HyperDrop.entrance(ori_train, after_pca, hla_dic, dic_inventory, top_k=0.5)
external_graphs, external_graph_labels = HyperGraph_Constructor_DHT_Cluster.entrance(ori_train_Dengue, after_pca, hla_dic, dic_inventory, n_clusters=5)

# Initialize the generator for dual hypergraphs
generator = PaddedGraphGenerator(graphs=graphs)

count_graph_labels = ori_train_Dengue['immunogenicity']
count_graph_labels.value_counts().to_frame()

assert len(external_graphs) == len(external_graph_labels), "Mismatch in number of graphs and labels."

In [ ]:
# Step 1: Load external datasets
from stellargraph import StellarGraph
import pandas as pd
import numpy as np

# Ensure data integrity
assert len(external_graphs) == len(external_graph_labels), "Mismatch in graphs and labels."

# Step 2: Preprocess external graphs (if necessary)
node_feature_size = graphs[0].node_features().shape[1]

def preprocess_graph(graph):
    features = graph.node_features()
    if features.shape[1] != node_feature_size:
        padded_features = np.zeros((features.shape[0], node_feature_size))
        padded_features[:, :features.shape[1]] = features
        graph = StellarGraph(graph.to_networkx(), node_features=padded_features)
    return graph

external_graphs = [preprocess_graph(g) for g in external_graphs]

# Step 3: Create data generators
from stellargraph.mapper import PaddedGraphGenerator

external_generator = PaddedGraphGenerator(graphs=external_graphs)
external_index = np.arange(len(external_graphs))

external_gen = external_generator.flow(
    external_index,
    targets=external_graph_labels.values,
    batch_size=50,
    symmetric_normalization=False,
)

# Step 4: Predict on external data
external_predictions_proba = model.predict(external_gen).ravel()

# Adjust the threshold based on precision-recall curve
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(external_graph_labels.values, external_predictions_proba)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold:.4f}")

# Apply the optimal threshold
external_predictions = (external_predictions_proba > optimal_threshold).astype(int)

# Step 5: Compute and display metrics
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, average_precision_score

external_y_true = external_graph_labels.values
external_accuracy = accuracy_score(external_y_true, external_predictions)
external_mcc = matthews_corrcoef(external_y_true, external_predictions)
external_roc_auc = roc_auc_score(external_y_true, external_predictions_proba)
external_pr_auc = average_precision_score(external_y_true, external_predictions_proba)

print(f"Accuracy on External Data: {external_accuracy:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {external_mcc:.4f}")
print(f"ROC AUC Score: {external_roc_auc:.4f}")
print(f"Precision-Recall AUC: {external_pr_auc:.4f}")

# Step 6: Additional metrics and visualization
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("\nClassification Report on External Data:")
print(classification_report(external_y_true, external_predictions))

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['Negative', 'Positive'],
        yticklabels=['Negative', 'Positive']
    )
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix on External Data')
    plt.show()

# Plot the confusion matrix
plot_confusion_matrix(external_y_true, external_predictions)

# Plot ROC Curve
def plot_roc_curve(y_true, y_proba):
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve on External Data')
    plt.legend(loc='lower right')
    plt.show()

plot_roc_curve(external_y_true, external_predictions_proba)

# Plot Precision-Recall Curve
def plot_precision_recall_curve(y_true, y_proba):
    from sklearn.metrics import precision_recall_curve, auc
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    pr_auc = auc(recall, precision)

    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AUC = {pr_auc:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve on External Data')
    plt.legend(loc='lower left')
    plt.show()

plot_precision_recall_curve(external_y_true, external_predictions_proba)


In [ ]:
# Covid convalescent

In [ ]:

# ori_train = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\remove0123_sample100.csv')
ori_train_Dengue = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\sars_cov_2_result.csv')
ori_train_Dengue_bk = ori_train_Dengue
hla = pd.read_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\hla2paratopeTable_aligned.txt',sep='\t')
after_pca = np.loadtxt('D:\\PhD\\GraphImmuno_NewResults\\data\\after_pca.txt')

hla_dic = hla_df_to_dic(hla)
inventory = list(hla_dic.keys())
dic_inventory = dict_inventory(inventory)

ori_train_Dengue['immunogenicity'] = ori_train_Dengue['immunogenicity-con']

# ori_train['immunogenicity'], ori_train['potential'] = ori_train_external['potential'], ori_train_external['immunogenicity']

# Create graphs and labels using the new HyperGraph_Constructor
#graphs, graph_labels = HyperGraph_Constructor_DHT.entrance(ori_train, after_pca, hla_dic, dic_inventory)
#graphs, graph_labels = HyperGraph_Constructor_DHT_HyperDrop.entrance(ori_train, after_pca, hla_dic, dic_inventory, top_k=0.5)
external_graphs, external_graph_labels = HyperGraph_Constructor_DHT_Cluster.entrance(ori_train_Dengue, after_pca, hla_dic, dic_inventory, n_clusters=5)

# Initialize the generator for dual hypergraphs
generator = PaddedGraphGenerator(graphs=graphs)

count_graph_labels = ori_train_Dengue['immunogenicity']
count_graph_labels.value_counts().to_frame()

assert len(external_graphs) == len(external_graph_labels), "Mismatch in number of graphs and labels."

In [ ]:
# Step 1: Load external datasets
from stellargraph import StellarGraph
import pandas as pd
import numpy as np

# Ensure data integrity
assert len(external_graphs) == len(external_graph_labels), "Mismatch in graphs and labels."

# Step 2: Preprocess external graphs (if necessary)
# Example preprocessing (ensure node features have the correct size)
node_feature_size = graphs[0].node_features().shape[1]

def preprocess_graph(graph):
    features = graph.node_features()
    if features.shape[1] != node_feature_size:
        padded_features = np.zeros((features.shape[0], node_feature_size))
        padded_features[:, :features.shape[1]] = features
        graph = StellarGraph(graph.to_networkx(), node_features=padded_features)
    return graph

external_graphs = [preprocess_graph(g) for g in external_graphs]

# Step 3: Create data generators
from stellargraph.mapper import PaddedGraphGenerator

external_generator = PaddedGraphGenerator(graphs=external_graphs)
external_index = np.arange(len(external_graphs))

external_gen = external_generator.flow(
    external_index,
    targets=external_graph_labels.values,
    batch_size=50,  # Adjust batch size if needed
    symmetric_normalization=False,
)

# Step 4: Predict on external data
external_predictions_proba = model.predict(external_gen).ravel()
external_predictions = (external_predictions_proba > 0.5).astype(int)

# Step 5: Compute and display metrics
from sklearn.metrics import accuracy_score, matthews_corrcoef

external_y_true = external_graph_labels.values
external_accuracy = accuracy_score(external_y_true, external_predictions)
external_mcc = matthews_corrcoef(external_y_true, external_predictions)

print(f"Accuracy on External Data: {external_accuracy:.4f}")
print(f"Matthews Correlation Coefficient (MCC) on External Data: {external_mcc:.4f}")

# Step 6: Additional metrics and visualization
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("\nClassification Report on External Data:")
print(classification_report(external_y_true, external_predictions))

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['Negative', 'Positive'],
        yticklabels=['Negative', 'Positive']
    )
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix on External Data')
    plt.show()

# Plot the confusion matrix
plot_confusion_matrix(external_y_true, external_predictions)


In [ ]:
# Step 1: Load external datasets
from stellargraph import StellarGraph
import pandas as pd
import numpy as np

# Ensure data integrity
assert len(external_graphs) == len(external_graph_labels), "Mismatch in graphs and labels."

# Step 2: Preprocess external graphs (if necessary)
node_feature_size = graphs[0].node_features().shape[1]

def preprocess_graph(graph):
    features = graph.node_features()
    if features.shape[1] != node_feature_size:
        padded_features = np.zeros((features.shape[0], node_feature_size))
        padded_features[:, :features.shape[1]] = features
        graph = StellarGraph(graph.to_networkx(), node_features=padded_features)
    return graph

external_graphs = [preprocess_graph(g) for g in external_graphs]

# Step 3: Create data generators
from stellargraph.mapper import PaddedGraphGenerator

external_generator = PaddedGraphGenerator(graphs=external_graphs)
external_index = np.arange(len(external_graphs))

external_gen = external_generator.flow(
    external_index,
    targets=external_graph_labels.values,
    batch_size=50,
    symmetric_normalization=False,
)

# Step 4: Predict on external data
external_predictions_proba = model.predict(external_gen).ravel()

# Adjust the threshold based on precision-recall curve
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(external_graph_labels.values, external_predictions_proba)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold:.4f}")

# Apply the optimal threshold
external_predictions = (external_predictions_proba > optimal_threshold).astype(int)

# Step 5: Compute and display metrics
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, average_precision_score

external_y_true = external_graph_labels.values
external_accuracy = accuracy_score(external_y_true, external_predictions)
external_mcc = matthews_corrcoef(external_y_true, external_predictions)
external_roc_auc = roc_auc_score(external_y_true, external_predictions_proba)
external_pr_auc = average_precision_score(external_y_true, external_predictions_proba)

print(f"Accuracy on External Data: {external_accuracy:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {external_mcc:.4f}")
print(f"ROC AUC Score: {external_roc_auc:.4f}")
print(f"Precision-Recall AUC: {external_pr_auc:.4f}")

# Step 6: Additional metrics and visualization
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("\nClassification Report on External Data:")
print(classification_report(external_y_true, external_predictions))

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['Negative', 'Positive'],
        yticklabels=['Negative', 'Positive']
    )
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix on External Data')
    plt.show()

# Plot the confusion matrix
plot_confusion_matrix(external_y_true, external_predictions)

# Plot ROC Curve
def plot_roc_curve(y_true, y_proba):
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve on External Data')
    plt.legend(loc='lower right')
    plt.show()

plot_roc_curve(external_y_true, external_predictions_proba)

# Plot Precision-Recall Curve
def plot_precision_recall_curve(y_true, y_proba):
    from sklearn.metrics import precision_recall_curve, auc
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    pr_auc = auc(recall, precision)

    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AUC = {pr_auc:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve on External Data')
    plt.legend(loc='lower left')
    plt.show()

plot_precision_recall_curve(external_y_true, external_predictions_proba)


In [ ]:
# Covid unexposed

In [ ]:

# ori_train = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\remove0123_sample100.csv')
ori_train_Dengue = pd.read_csv('D:\\PhD\Subgraph_Sampling_SLOT_GNN\\Data\\sars_cov_2_result.csv')
ori_train_Dengue_bk = ori_train_Dengue
hla = pd.read_csv('D:\\PhD\\GraphImmuno_NewResults\\data\\hla2paratopeTable_aligned.txt',sep='\t')
after_pca = np.loadtxt('D:\\PhD\\GraphImmuno_NewResults\\data\\after_pca.txt')

hla_dic = hla_df_to_dic(hla)
inventory = list(hla_dic.keys())
dic_inventory = dict_inventory(inventory)

ori_train_Dengue['immunogenicity'] = ori_train_Dengue['immunogenicity-un']

# ori_train['immunogenicity'], ori_train['potential'] = ori_train_external['potential'], ori_train_external['immunogenicity']

# Create graphs and labels using the new HyperGraph_Constructor
#graphs, graph_labels = HyperGraph_Constructor_DHT.entrance(ori_train, after_pca, hla_dic, dic_inventory)
#graphs, graph_labels = HyperGraph_Constructor_DHT_HyperDrop.entrance(ori_train, after_pca, hla_dic, dic_inventory, top_k=0.5)
external_graphs, external_graph_labels = HyperGraph_Constructor_DHT_Cluster.entrance(ori_train_Dengue, after_pca, hla_dic, dic_inventory, n_clusters=5)

# Initialize the generator for dual hypergraphs
generator = PaddedGraphGenerator(graphs=graphs)

count_graph_labels = ori_train_Dengue['immunogenicity']
count_graph_labels.value_counts().to_frame()

assert len(external_graphs) == len(external_graph_labels), "Mismatch in number of graphs and labels."

In [ ]:
# Step 1: Load external datasets
from stellargraph import StellarGraph
import pandas as pd
import numpy as np

# Ensure data integrity
assert len(external_graphs) == len(external_graph_labels), "Mismatch in graphs and labels."

# Step 2: Preprocess external graphs (if necessary)
node_feature_size = graphs[0].node_features().shape[1]

def preprocess_graph(graph):
    features = graph.node_features()
    if features.shape[1] != node_feature_size:
        padded_features = np.zeros((features.shape[0], node_feature_size))
        padded_features[:, :features.shape[1]] = features
        graph = StellarGraph(graph.to_networkx(), node_features=padded_features)
    return graph

external_graphs = [preprocess_graph(g) for g in external_graphs]

# Step 3: Create data generators
from stellargraph.mapper import PaddedGraphGenerator

external_generator = PaddedGraphGenerator(graphs=external_graphs)
external_index = np.arange(len(external_graphs))

external_gen = external_generator.flow(
    external_index,
    targets=external_graph_labels.values,
    batch_size=50,
    symmetric_normalization=False,
)

# Step 4: Predict on external data
external_predictions_proba = model.predict(external_gen).ravel()

# Adjust the threshold based on precision-recall curve
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(external_graph_labels.values, external_predictions_proba)
f1_scores = 2 * (precision * recall) / (precision + recall)
optimal_idx = np.argmax(f1_scores)
optimal_threshold = thresholds[optimal_idx]
print(f"Optimal Threshold: {optimal_threshold:.4f}")

# Apply the optimal threshold
external_predictions = (external_predictions_proba > optimal_threshold).astype(int)

# Step 5: Compute and display metrics
from sklearn.metrics import accuracy_score, matthews_corrcoef, roc_auc_score, average_precision_score

external_y_true = external_graph_labels.values
external_accuracy = accuracy_score(external_y_true, external_predictions)
external_mcc = matthews_corrcoef(external_y_true, external_predictions)
external_roc_auc = roc_auc_score(external_y_true, external_predictions_proba)
external_pr_auc = average_precision_score(external_y_true, external_predictions_proba)

print(f"Accuracy on External Data: {external_accuracy:.4f}")
print(f"Matthews Correlation Coefficient (MCC): {external_mcc:.4f}")
print(f"ROC AUC Score: {external_roc_auc:.4f}")
print(f"Precision-Recall AUC: {external_pr_auc:.4f}")

# Step 6: Additional metrics and visualization
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

print("\nClassification Report on External Data:")
print(classification_report(external_y_true, external_predictions))

def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(7, 5))
    sns.heatmap(
        cm,
        annot=True,
        fmt='d',
        cmap='Blues',
        xticklabels=['Negative', 'Positive'],
        yticklabels=['Negative', 'Positive']
    )
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.title('Confusion Matrix on External Data')
    plt.show()

# Plot the confusion matrix
plot_confusion_matrix(external_y_true, external_predictions)

# Plot ROC Curve
def plot_roc_curve(y_true, y_proba):
    from sklearn.metrics import roc_curve, auc
    fpr, tpr, _ = roc_curve(y_true, y_proba)
    roc_auc = auc(fpr, tpr)

    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([-0.05, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve on External Data')
    plt.legend(loc='lower right')
    plt.show()

plot_roc_curve(external_y_true, external_predictions_proba)

# Plot Precision-Recall Curve
def plot_precision_recall_curve(y_true, y_proba):
    from sklearn.metrics import precision_recall_curve, auc
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    pr_auc = auc(recall, precision)

    plt.figure()
    plt.plot(recall, precision, color='blue', lw=2, label=f'PR curve (AUC = {pr_auc:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve on External Data')
    plt.legend(loc='lower left')
    plt.show()

plot_precision_recall_curve(external_y_true, external_predictions_proba)


In [ ]:
type(graphs[0])

In [ ]:
# Unsupervised Learning

In [ ]:
!pip install graph2vec

In [ ]:
!pip install karateclub

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx
from karateclub import Graph2Vec

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE



# Convert StellarGraph objects to NetworkX graphs and relabel nodes
nx_graphs = []
for g in graphs:
    # Convert to NetworkX graph
    nx_g = g.to_networkx()

    # Relabel nodes to consecutive integers starting from 0
    nx_g = nx.convert_node_labels_to_integers(nx_g, first_label=0, ordering='default')
    nx_graphs.append(nx_g)

# Instantiate Graph2Vec model
graph2vec_model = Graph2Vec(dimensions=128, workers=4, epochs=10, min_count=1)

# Fit the model to the list of NetworkX graphs
print("Fitting Graph2Vec model...")
graph2vec_model.fit(nx_graphs)

# Get the embeddings for each graph
graph_embeddings = graph2vec_model.get_embedding()

# Apply KMeans clustering
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(graph_embeddings)

# Analyze clusters
unique, counts = np.unique(clusters, return_counts=True)
cluster_info = dict(zip(unique, counts))
print("Cluster Distribution:")
for cluster_id, size in cluster_info.items():
    print(f"Cluster {cluster_id}: {size} graphs")

# Visualize with PCA
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(graph_embeddings)

plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=reduced_embeddings[:, 0],
    y=reduced_embeddings[:, 1],
    hue=clusters,
    palette="viridis",
    s=50,
    alpha=0.8
)
plt.title("Graph Clustering Visualization with PCA")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title='Cluster')
plt.show()

# Visualize with t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
tsne_embeddings = tsne.fit_transform(graph_embeddings)

plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=tsne_embeddings[:, 0],
    y=tsne_embeddings[:, 1],
    hue=clusters,
    palette="viridis",
    s=50,
    alpha=0.8
)
plt.title("Graph Clustering Visualization with t-SNE")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend(title='Cluster')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx
from stellargraph import StellarGraph
from karateclub import Graph2Vec

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE


In [ ]:
nx_graphs = []
for g in graphs:
    # Convert to NetworkX graph
    nx_g = g.to_networkx()

    # Relabel nodes to consecutive integers starting from 0
    nx_g = nx.convert_node_labels_to_integers(nx_g, first_label=0, ordering='default')
    nx_graphs.append(nx_g)
# Instantiate Graph2Vec model
graph2vec_model = Graph2Vec(dimensions=128, workers=4, epochs=10, min_count=1)

# Fit the model to the list of NetworkX graphs
print("Fitting Graph2Vec model...")
graph2vec_model.fit(nx_graphs)

# Get the embeddings for each graph
graph_embeddings = graph2vec_model.get_embedding()

# Define the number of clusters
n_clusters = 5

# Apply KMeans clustering
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(graph_embeddings)

# Analyze clusters
unique, counts = np.unique(clusters, return_counts=True)
cluster_info = dict(zip(unique, counts))
print("Cluster Distribution:")
for cluster_id, size in cluster_info.items():
    print(f"Cluster {cluster_id}: {size} graphs")
# Reduce dimensionality with PCA
pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(graph_embeddings)

# Plot the clusters
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=reduced_embeddings[:, 0],
    y=reduced_embeddings[:, 1],
    hue=clusters,
    palette="viridis",
    s=50,
    alpha=0.8
)
plt.title("Graph Clustering Visualization with PCA")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title='Cluster')
plt.show()


In [ ]:
# Reduce dimensionality with t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=1000, random_state=42)
tsne_embeddings = tsne.fit_transform(graph_embeddings)

# Plot the clusters
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=tsne_embeddings[:, 0],
    y=tsne_embeddings[:, 1],
    hue=clusters,
    palette="viridis",
    s=50,
    alpha=0.8
)
plt.title("Graph Clustering Visualization with t-SNE")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.legend(title='Cluster')
plt.show()


In [ ]:
#Graph Kernel Methods

In [ ]:
!pip install grakel

In [ ]:
#3. Subgraph/Motif Mining

In [ ]:
import networkx as nx

# Find cliques (fully connected subgraphs) in each graph
for i, nx_g in enumerate(nx_graphs):
    cliques = list(nx.find_cliques(nx_g))
    print(f"Graph {i} has {len(cliques)} cliques")

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Assuming nx_graphs is your list of NetworkX graphs
# Ensure that your graphs are simple graphs (no multiple edges)
for i, nx_g in enumerate(nx_graphs):
    if isinstance(nx_g, nx.MultiGraph) or isinstance(nx_g, nx.MultiDiGraph):
        nx_g = nx.Graph(nx_g)  # Convert to simple graph
        nx_graphs[i] = nx_g

# Calculate clique sizes
clique_sizes = []
for i, nx_g in enumerate(nx_graphs):
    # Find all maximal cliques in the graph
    cliques = list(nx.find_cliques(nx_g))
    # Get the sizes of each clique and extend the list
    clique_sizes.extend([len(clique) for clique in cliques])
    print(f"Graph {i+1} has {len(cliques)} cliques")

# Plot the distribution of clique sizes
plt.figure(figsize=(10, 6))
plt.hist(clique_sizes, bins=range(1, max(clique_sizes)+2), align='left', edgecolor='black', rwidth=0.8)
plt.title('Distribution of Clique Sizes Across All Graphs')
plt.xlabel('Clique Size')
plt.ylabel('Frequency')
plt.xticks(range(1, max(clique_sizes)+1))
plt.show()


In [ ]:
import pandas as pd
import seaborn as sns

# Create a DataFrame from the clique sizes
clique_sizes_df = pd.DataFrame({'Clique Size': clique_sizes})

# Count the frequency of each clique size
clique_size_counts = clique_sizes_df['Clique Size'].value_counts().sort_index()

# Plot the bar chart
plt.figure(figsize=(10, 6))
sns.barplot(x=clique_size_counts.index, y=clique_size_counts.values, color='skyblue')
plt.title('Distribution of Clique Sizes Across All Graphs')
plt.xlabel('Clique Size')
plt.ylabel('Frequency')
plt.show()


In [ ]:
import networkx as nx

# Select a graph to visualize
graph_index = 0  # Change to the index of the graph you want to visualize
nx_g = nx_graphs[graph_index]

# Find cliques
cliques = list(nx.find_cliques(nx_g))

# Get the largest cliques
max_clique_size = max(len(clique) for clique in cliques)
largest_cliques = [clique for clique in cliques if len(clique) == max_clique_size]

# Visualize the graph highlighting the largest cliques
pos = nx.spring_layout(nx_g)
plt.figure(figsize=(12, 8))
nx.draw_networkx_edges(nx_g, pos, alpha=0.5)
nx.draw_networkx_nodes(nx_g, pos, node_color='lightblue', node_size=300)
nx.draw_networkx_labels(nx_g, pos, font_size=10)

# Highlight nodes in the largest cliques
for clique in largest_cliques:
    nx.draw_networkx_nodes(nx_g, pos, nodelist=clique, node_color='red', node_size=300)

plt.title(f'Graph {graph_index+1} with Largest Cliques Highlighted')
plt.axis('off')
plt.show()


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Assuming nx_graphs is your list of NetworkX graphs
# Ensure that your graphs are simple graphs (no multiple edges)
for i, nx_g in enumerate(nx_graphs):
    if isinstance(nx_g, nx.MultiGraph) or isinstance(nx_g, nx.MultiDiGraph):
        nx_g = nx.Graph(nx_g)  # Convert to simple graph
        nx_graphs[i] = nx_g

# Calculate clique sizes
clique_sizes = []
for i, nx_g in enumerate(nx_graphs):
    # Find all maximal cliques in the graph
    cliques = list(nx.find_cliques(nx_g))
    # Get the sizes of each clique and extend the list
    clique_sizes.extend([len(clique) for clique in cliques])
    print(f"Graph {i+1} has {len(cliques)} cliques")




In [ ]:
# Plot the distribution of clique sizes
plt.figure(figsize=(10, 6))
plt.hist(clique_sizes, bins=range(1, max(clique_sizes)+2), align='left', edgecolor='black', rwidth=0.8)
plt.title('Distribution of Clique Sizes Across All Graphs')
plt.xlabel('Clique Size')
plt.ylabel('Frequency')
plt.xticks(range(1, max(clique_sizes)+1))
plt.show()

In [ ]:
#4. Graph Generative Models

In [ ]:
#5. Clustering Based on Graph Features

In [ ]:
# Extract features for each graph
import networkx as nx

graph_features = []

for nx_g in nx_graphs:
    # Convert MultiGraph to simple Graph
    if isinstance(nx_g, nx.MultiGraph) or isinstance(nx_g, nx.MultiDiGraph):
        nx_g = nx.Graph(nx_g)

    features = {}
    features['num_nodes'] = nx_g.number_of_nodes()
    features['num_edges'] = nx_g.number_of_edges()
    features['average_degree'] = sum(dict(nx_g.degree()).values()) / nx_g.number_of_nodes()
    features['density'] = nx.density(nx_g)
    features['average_clustering'] = nx.average_clustering(nx_g)
    # Add more features as needed

    graph_features.append(features)

# Convert to DataFrame
import pandas as pd
graph_features_df = pd.DataFrame(graph_features)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_features = scaler.fit_transform(graph_features_df)
from sklearn.cluster import KMeans

n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
clusters = kmeans.fit_predict(scaled_features)

# Add clusters to DataFrame
graph_features_df['cluster'] = clusters

# Reduce dimensionality for visualization
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
reduced_features = pca.fit_transform(scaled_features)

plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=reduced_features[:, 0],
    y=reduced_features[:, 1],
    hue=clusters,
    palette="viridis",
    s=50,
    alpha=0.8
)
plt.title("Graph Clustering Based on Structural Features")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title='Cluster')
plt.show()


In [ ]:
#6. Node Embedding Learning Across Multiple Graphs

In [ ]:
!pip install node2vec

In [ ]:
from node2vec import Node2Vec

node_embeddings = []

for i, nx_g in enumerate(nx_graphs):
    print(f"Processing graph {i+1}/{len(nx_graphs)}")

    # Ensure the graph is connected and has enough nodes
    if nx_g.number_of_nodes() < 2:
        print(f"Graph {i+1} is too small. Skipping.")
        continue

    if not nx.is_connected(nx_g):
        # Take the largest connected component
        largest_cc = max(nx.connected_components(nx_g), key=len)
        nx_g = nx_g.subgraph(largest_cc).copy()

    try:
        # Create and fit Node2Vec model
        node2vec = Node2Vec(
            nx_g,
            dimensions=32,
            walk_length=10,
            num_walks=50,
            workers=1
        )
        model = node2vec.fit(window=5, min_count=1, batch_words=4)

        embeddings = model.wv
        node_embeddings.append(embeddings)
    except Exception as e:
        print(f"Error processing graph {i+1}: {e}")
        continue

    # Fit model
    model = node2vec.fit(window=10, min_count=1, batch_words=4)

    # Get embeddings as a dictionary {node_id: embedding_vector}
    embeddings = model.wv
    node_embeddings.append(embeddings)

    graph_embeddings = []

for embeddings in node_embeddings:
    # Aggregate node embeddings to create a graph embedding
    # You can take the mean of all node embeddings
    emb_vectors = np.array([embeddings[node_id] for node_id in embeddings.key_to_index])
    graph_emb = np.mean(emb_vectors, axis=0)
    graph_embeddings.append(graph_emb)

graph_embeddings = np.array(graph_embeddings)


In [ ]:
graph_embeddings

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans


In [ ]:
!pip install umap-learn


In [ ]:
!pip install importlib-metadata

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans

# Assuming graph_embeddings is your numpy array of shape (num_graphs, embedding_dim)

# Verify that graph_embeddings is a NumPy array
print(type(graph_embeddings))
print(graph_embeddings.shape)

# Proceed if it's a NumPy array
if isinstance(graph_embeddings, np.ndarray):
    # Clustering
    n_clusters = 5  # Adjust as needed
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(graph_embeddings)

    # PCA Visualization
    pca = PCA(n_components=2)
    graph_embeddings_pca = pca.fit_transform(graph_embeddings)

    plt.figure(figsize=(10, 8))
    sns.scatterplot(
        x=graph_embeddings_pca[:, 0],
        y=graph_embeddings_pca[:, 1],
        hue=clusters,
        palette='viridis',
        s=50,
        alpha=0.8
    )
    plt.title('Graph Embeddings with PCA and KMeans Clustering')
    plt.xlabel('Principal Component 1')
    plt.ylabel('Principal Component 2')
    plt.legend(title='Cluster')
    plt.show()
else:
    print("graph_embeddings is not a NumPy array. Please check your code.")


In [ ]:
#Using Graph Neural Networks for Graph Embeddings

In [ ]:
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

generator = PaddedGraphGenerator(graphs=graphs)

# Define the GCN model
gcn_model = GCNSupervisedGraphClassification(
    layer_sizes=[64, 64],
    activations=["relu", "relu"],
    generator=generator,
    dropout=0.5,
)

# Build the model
x_inp, x_out = gcn_model.in_out_tensors()
graph_embeddings = x_out  # This is the graph embedding

# Since we are doing unsupervised learning, we can use the embeddings directly
embedding_model = Model(inputs=x_inp, outputs=graph_embeddings)

# Generate embeddings for all graphs
graph_indices = np.arange(len(graphs))
embeddings = embedding_model.predict(generator.flow(graph_indices, batch_size=50), verbose=1)

# Now, embeddings can be used for clustering or visualization


In [ ]:
# Plot the clusters
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x=embeddings[:, 0],
    y=embeddings[:, 1],
    hue=clusters,
    palette="viridis",
    s=50,
    alpha=0.8
)
plt.title("Graph Neural Networks for Graph Embeddings")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.legend(title='Cluster')
plt.show()